# Tutorial 02 - Elastic Block

In [ ]:
import typing

In [ ]:
# import dolfinx
import gmsh
import multiphenicsx.io
import multiphenicsx.mesh

In [ ]:
# import rbnicsx

## 1. Mesh generation

In [ ]:
def generate_rectangular_subdomains(
    x: typing.List[float], y: typing.List[float], mesh_size: float
) -> typing.Tuple[int]:
    """Add points and lines that define rectangular subdomains with the provided coordinates."""
    points = [[gmsh.model.geo.addPoint(x_, y_, 0.0, mesh_size) for y_ in y] for x_ in x]
    horizontal_lines = [
        [gmsh.model.geo.addLine(points[i][j], points[i + 1][j]) for j in range(len(points[i]))]
        for i in range(len(points) - 1)]
    vertical_lines = [
        [gmsh.model.geo.addLine(points[i][j], points[i][j + 1]) for j in range(len(points[i]) - 1)]
        for i in range(len(points))]
    curve_loops = [
        [gmsh.model.geo.addCurveLoop([
            horizontal_lines[i][j], vertical_lines[i + 1][j],
            - horizontal_lines[i][j + 1], - vertical_lines[i][j]]) for j in range(len(points[i]) - 1)]
        for i in range(len(points) - 1)]
    boundaries = [
        [{
            "horizontal": (
                horizontal_lines[i][j] if j == 0 else
                horizontal_lines[i][j + 1] if j == len(points[i]) - 2 else None),
            "vertical": (
                vertical_lines[i][j] if i == 0 else
                vertical_lines[i + 1][j] if i == len(points) - 2 else None)
        } for j in range(len(points[i]) - 1)]
        for i in range(len(points) - 1)]
    subdomains = [
        [gmsh.model.geo.addPlaneSurface([curve_loops[i][j]]) for j in range(len(points[i]) - 1)]
        for i in range(len(points) - 1)]
    return subdomains, boundaries

In [ ]:
mesh_size = 5e-2

In [ ]:
gmsh.initialize()
gmsh.model.add("elastic_block")

In [ ]:
gmsh_subdomains, gmsh_boundaries = generate_rectangular_subdomains(
    [i / 3. for i in range(4)], [j / 3. for j in range(4)], mesh_size)

In [ ]:
gmsh.model.geo.synchronize()
for i in range(3):
    for j in range(3):
        gmsh.model.addPhysicalGroup(2, [gmsh_subdomains[i][j]], i + 3 * j + 1)
gmsh.model.addPhysicalGroup(1, [gmsh_boundaries[i][0]["horizontal"] for i in range(3)], 1)
[gmsh.model.addPhysicalGroup(1, [gmsh_boundaries[2][j]["vertical"]], j + 2) for j in range(3)]
gmsh.model.addPhysicalGroup(1, [gmsh_boundaries[i][2]["horizontal"] for i in range(3)], 5)
gmsh.model.addPhysicalGroup(1, [gmsh_boundaries[0][j]["vertical"] for j in range(3)], 6)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)